# PLAsTiCC Astronomical Classification Tutorial

## Introduction

In this tutorial, we will be using a simulated timeseries dataset from a new telescope, named the "Large Synoptic Survey Telescope", to predict the classes of astronomical objects. The timeseries measurements of brightness include the photon flux.

The photon flux is measured by capturing the intensity from six different passbands of the light spectrum. Images of the night sky are taken and differenced to determine how much the light changed between subsequent observations

PLAsTiCC stands for "Photometric LSST Astronomical Time Series Classification Challenge"


## Data

A comprehensive description of the dataset can be found [here](https://www.kaggle.com/c/PLAsTiCC-2018/data).

#### The columns in the training set contain the following fields:
- **object_id** [int32]: Unique id used to identify astronomical objects.
- **mjd** [float64]: the time in Modified Julian Date (MJD) of the observation.
- **passband** [int8]: The specific LSST passband integer, such that u, g, r, i, z, Y = 0, 1, 2, 3, 4, 5 in which it was viewed.
- **flux** [float32]: the measured flux (brightness) in the passband of observation as listed in the passband column.
- **flux_err** [float32]: the uncertainty on the measurement of the flux measurement.

#### Metadata columns in the training set contain the following fields:
- **object_id** [int32]: unique object identifier.
- **ddf** [boolean]: A flag to identify the object as coming from the DDF survey area (with value DDF = 1 for the DDF, DDF = 0 for the WFD survey). Note that while the DDF fields are contained within the full WFD survey area, the DDF fluxes have significantly smaller uncertainties. 
- **hostgal_specz** [float32]: the spectroscopic redshift of the source. This is an extremely accurate measure of redshift, available for the training set and a small fraction of the test set.
- **hostgal_photoz** [float32]: The photometric redshift of the host galaxy of the astronomical source. While this is meant to be a proxy for hostgal_specz, there can be large differences between the two and should be regarded as a far less accurate version of hostgal_specz.
- **hostgal_photoz_err** [float32]: The uncertainty on the hostgal_photoz based on LSST survey projections.
- **distmod** [float32]: The distance to the source calculated from hostgal_photoz and using general relativity.
- **mwebv** [float32]: MW E(B-V). this ‘extinction’ of light is a property of the Milky Way (MW) dust along the line of sight to the astronomical source, and is thus a function of the sky coordinates of the source ra, decl. This is used to determine a passband dependent dimming and redenning of light from astronomical sources as described in subsection 2.1, and based on the Schlafly et al. (2011) and Schlegel et al. (1998) dust models.
- **target** [int8]: The class of the astronomical source. This is provided in the training data. Correctly determining the target (correctly assigning classification probabilities to the objects) is the ‘goal’ of the classification challenge for the test data. Note that there is one class in the test set that does not occur in the training set: class_99 serves as an "other" class for objects that don't belong in any of the 14 classes in the training set.

#### Caveats / Additional Challenges

- There gaps in the times at which passband observations were taken, sometimes many days apart
- Galactic vs extragalactic: The given redshift for objects in our own Milky Way
galaxy is given as zero.
- Due to statistical fluctuations (of e.g. the sky brightness) and the
way the brightness is estimated, the flux may be negative for dim sources, where
the true flux is close to zero. Additionally, if the pre-survey image actually contains a
flux brighter than its true ‘zero’, this can lead to a negative flux when the difference
is computed.


This introduction was adapted from [The Photometric LSST Astronomical
Time-series Classification Challenge, The PLaSTiCC Team et al, 2018](https://arxiv.org/pdf/1810.00001.pdf). Refer to this reference for more details. 


## Solution

This tutorial will explore the [8th place](https://www.kaggle.com/c/PLAsTiCC-2018/leaderboard) solution to the challenge, which uses the RAPIDS open-source ecosystem of data science tools for GPU-accelerated feature engineering and model building. It has been simplified from the [original solution](https://devblogs.nvidia.com/make-sense-universe-rapids-ai/) in order to fit within a KDD tutorial session. 

The diagram below outlines the full end-to-end solution, which uses RAPIDS cuDF to engineer features for training XGBoost, MLP, and Bidirectional (attentional) RNN models. This tutorial will include the feature engineering and training processes for the XGBoost and RNN models. The training of the MLP model and the [model stacking](https://towardsdatascience.com/automate-stacking-in-python-fc3e7834772e) are left as future work for the students.

<img src="https://devblogs.nvidia.com/wp-content/uploads/2019/02/1tv0fuN-usAsQLJzTj1zSLQ.png" width=700/>

For the sake of time, we will be implementing a simplified version of the end-to-end Kaggle solution. In this simplified version, we will
1. engineer timeseries features by converting timeseries into fixed-length vectors using a pre-trained RNN model
2. engineer aggregate features by building statistical summaries from the input data, and 
3. train an XGBoost classifier using both our fixed-length timeseries vectors & our aggregated features to classify the data in the test set

<hr/>

### This tutorial is broken down into two parts:

#### Part 1 - Training RNN Timeseries Embeddings
- **Part 1.1:** Feature engineering for RNN feature extraction
- **Part 1.2:** Map timeseries features from Part 1.1 into fixed-length vectors using pre-trained RNN

#### Part 2 - Training an XGBoost Classifier
- **Part 2.1:** Engineer features by aggregating columns across observations in the input dataset
- **Part 2.2:** Use features from parts *1.2* and *2.1* to train and evaluate an XGBoost classifier

## Conclusion

By the end of this tutorial, the student should feel comfortable performing common data processing tasks on GPUs using cuDF, training machine learning models with XGBoost, and integrating cuDF-built datasets into popular deep learning frameworks like [Tensorflow](https://www.tensorflow.org/). 

## Problem (In More Detail)


### The Large Synoptic Survey Telescope (LSST)

The PLAsTiCC team is responsible for the Large Synoptic Survey Telescope (LSST), and has plans of revolutionizing the field of astronomy. This telescope will enable astronomers to discover 10 to 100 times the amount of astronomical sources in the night sky than we have ever been able to measure before.

The LSST was under construction in a mountain in Chile during the time of the Kaggle competition, but will be operational in 2019. When complete, it will be 8-meters long and contain a 3-billion pixel camera. The dataset in this competition contains simulated timeseries for different objects in the sky, based on observations one might expect to see from the real telescope.


### Light curves

The LSST measures *light curves*, which are an object's brightness as a function of time. These light curves are captured by taking measurements of each object's photon flux in six different astronomical light filters, called passbands. The passbands include the ultra-violet, optical, and infrared regions of the light spectrum. The mode used to collect measures in the LSST is referred to as *photometry*.

The light curves result from taking two images in the same region of the sky on different nights and subtracting them. This differencing procedure is able to capture both moving objects, like asteroids (which are removed), and objects that stay in the same position but vary in brightness. The flux may decrease or increase with time, depending on when the object first exploded or brightened. 


### Objective

Astronomical objects are driven by different physical processes and astronomers separate these objects into different astronomical classes. The manner in which the flux changes is a good indicator of the object's class. The dataset in this competition contains a set of training light curves (time series), which contain labels for 14 different object classes. The testing dataset is unlabeled, and will contain objects from 15 different classes. The goal is to classify the astronomical objects into these 15 different classes.
